# Requirement
install allennlp - `pip install allennlp` or https://github.com/allenai/allennlp

install tqdm - `pip install tqdm`

In [1]:
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from tqdm import tqdm

# Model word base

In [2]:
archive = load_archive('./load/model1/model.tar.gz',cuda_device=-1)

In [3]:
predictor = Predictor.from_archive(archive,"seq2seq")

In [4]:
predictor.predict("Are you sure ?")["predicted_tokens"]

['Sure', '?']

# Model character base

In [5]:
archive2 = load_archive('./load/model2/model.tar.gz',cuda_device=-1)

In [6]:
predictor2 = Predictor.from_archive(archive2,"seq2seq")

In [7]:
predictor2.predict("Are you sure ?")["predicted_tokens"]

['Y', 'o', 'u', ' ', 's', 'u', 'r', 'e', ' ', '?']

# Test Predict

In [8]:
def _predict(sentence):
    print("Original sentence : {}".format(sentence))
    print("Word base model   : {}".format(" ".join(predictor.predict(sentence)["predicted_tokens"])))
    print("Char base model   : {}".format("".join(predictor2.predict(sentence)["predicted_tokens"])))

In [9]:
sentence="Show me the money!"

In [10]:
_predict(sentence)

Original sentence : Show me the money!
Word base model   : Show me the money .
Char base model   : Show me the money .


# Evalutate

In [11]:
with open("./load/en-test.txt","r") as f:
    data = f.readlines()

In [12]:
from nltk.translate import bleu_score

In [13]:
test = []
for line in data:
    temp=line.split("\t")
    test.append((temp[1],temp[2]))

In [17]:
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

smoothie = SmoothingFunction().method4
splitter = SpacyWordSplitter()

## Evaluate word model

In [18]:
bleu_1=[]
for line in tqdm(test):
    bleu_1.append(bleu(
        [[i.text for i in splitter.split_words(line[1])]],
        predictor.predict(line[0])["predicted_tokens"],
        smoothing_function=smoothie
    ))

100%|██████████| 1445/1445 [01:23<00:00, 17.32it/s]


In [25]:
print("Max : {:>6.2f}".format(max(bleu_1)*100))
print("Min : {:>6.2f}".format(min(bleu_1)*100))
print("Sum : {:>6.2f}".format(sum(bleu_1)/len(test)*100))

Max : 100.00
Min :   0.00
Sum :  19.03


## Evaluate char model

In [26]:
bleu_2=[]
for line in tqdm(test):
    bleu_2.append(bleu(
        [[i.text for i in splitter.split_words(line[1])]],
        "".join(predictor2.predict(line[0])["predicted_tokens"]).split(" "),
        smoothing_function=smoothie
    ))

100%|██████████| 1445/1445 [01:05<00:00, 22.16it/s]


In [27]:
print("Max : {:>6.2f}".format(max(bleu_2)*100))
print("Min : {:>6.2f}".format(min(bleu_2)*100))
print("Sum : {:>6.2f}".format(sum(bleu_2)/len(test)*100))

Max : 100.00
Min :   0.00
Sum :  17.05
